# 401 Assignment 1

## 1 Tuning Hyper-parameters

In [221]:
#imports
import pandas as pd
from sklearn import datasets, tree
from sklearn.model_selection import train_test_split, KFold
from IPython.display import display
from sklearn.preprocessing import StandardScaler

Several helper functions were created:
- get_average 
 - Simply returns the average result of a list
- scale_data
 - A small wrapper function for scaling data appropriately using sklearn's StandardScaler
- is_improvement
 - A function that calculates if the next iteration of the max_depth is a valid improvement. The formula requires that for any 1 increase in the average number of nodes, there should be at least a 1% in accuracy.

In [222]:
# helper functions

def get_average(arr):#returns the average of a python list
    s = 0
    for a in arr:
        s = s + a
    avg = s/len(arr)
    return avg

def scale_data(xtrain, xtest):#scales data using sklearns standard scaler
    feature_scaler = StandardScaler()
    return feature_scaler.fit_transform(xtrain), feature_scaler.transform(xtest)

def is_improvement(prev_accuracy, prev_num_nodes, current_accuracy, current_num_nodes):
    if (prev_accuracy is 0) and (prev_num_nodes is 0):
        return True
    prev_accuracy = prev_accuracy*100
    current_accuracy = current_accuracy*100
    accuracy_improvement = current_accuracy - prev_accuracy
    if accuracy_improvement<=0:
        return False
    node_delta = current_num_nodes - prev_num_nodes
    if accuracy_improvement < node_delta:
        return False
    return True
    

The k-fold cross validation was contained within a method.<br/>
This method takes the training data, as well as some optional parameters relating to the k-fold itself <br/>
The method works through max depths of 1 to 15<br/>
to achieve this the method runs a for loop through each depth in which it defines a new DecisionTreeClasifier. This DT is then KFolded and each set of test and training data within the k-fold is fitted to the tree, with the accuracy and number of nodes recorded. <br/>
For each max depth this is average and used to check if the classifier is a valid improvement.<br>
The method finally displays a dataframe and returns the best classifier

In [223]:
#KFold CV Method
def run_KFold_CV(XTrainKF, yTrainKF, n_splits=6, random_state = 0, shuffle = False):
    df = pd.DataFrame(columns=['Max_Tree_Depth', 'Average_Accuracy', 'Average_Node_Count'])
    depths = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
    prev_avg_accuracy = 0
    prev_num_nodes = 0
    for depth in depths:
        kf_tree = tree.DecisionTreeClassifier(max_depth=depth)
        kf = KFold(n_splits=n_splits, random_state=random_state, shuffle=shuffle)
        accuracy = []
        num_nodes = []
        for train_index, test_index in kf.split(XTrainKF):
            x_train, x_test = XTrainKF[train_index], XTrainKF[test_index]
            y_train, y_test = yTrainKF[train_index], yTrainKF[test_index]
            kf_tree = kf_tree.fit(x_train, y_train)
            predictions = kf_tree.predict(x_test)
            accuracy.append(kf_tree.score(x_test, y_test))
            num_nodes.append(kf_tree.tree_.node_count)
            acc_avg = get_average(accuracy)
            num_nodes_avg = get_average(num_nodes)
        df = df.append({'Max_Tree_Depth':depth, 'Average_Accuracy':acc_avg, 'Average_Node_Count': num_nodes_avg}, ignore_index=True)
        if is_improvement(prev_avg_accuracy, prev_num_nodes, acc_avg, num_nodes_avg):
            prev_avg_accuracy, prev_num_nodes = acc_avg, num_nodes_avg
            best_depth = depth
            best_tree = kf_tree
    display(df)
    print("The best classifier was found with depth ", best_depth)
    return best_tree

def test_best_tree(best_tree, x_test, y_test):
    predictions = best_tree.predict(x_test)
    accuracy = best_tree.score(x_test, y_test)
    print("The best classifier achieved an accuracy of ", accuracy, " on the test data")
 

### Iris plant recognition

In [224]:
# Iris plant recognition  
iris = datasets.load_iris()
iris_X = iris.data
iris_Y = iris.target

iris_X_train, iris_X_test, iris_y_train, iris_y_test = train_test_split(iris_X, iris_Y, test_size=0.3, random_state=0)
iris_X_train, iris_X_test = scale_data(iris_X_train, iris_X_test)

iris_best_tree = run_KFold_CV(iris_X_train, iris_y_train)
test_best_tree(iris_best_tree, iris_X_test, iris_y_test)

,Max_Tree_Depth,Average_Accuracy,Average_Node_Count
0,1.0,0.611111,3.000000
1,2.0,0.924292,5.000000
2,3.0,0.961874,8.333333
3,4.0,0.942810,11.333333
4,5.0,0.942810,11.333333
5,6.0,0.942810,11.666667
6,7.0,0.942810,11.666667
7,8.0,0.942810,11.666667
8,9.0,0.942810,11.666667
9,10.0,0.942810,11.666667


The best classifier was found with depth  3
The best classifier achieved an accuracy of  0.9777777777777777  on the test data


### Breast cancer diagnosis

In [225]:
# Breast cancer diagnosis
breast_cancer = datasets.load_breast_cancer()
breast_cancer_X = breast_cancer.data
breast_cancer_Y = breast_cancer.target

breast_cancer_X_train, breast_cancer_X_test, breast_cancer_y_train, breast_cancer_y_test = train_test_split(breast_cancer_X, breast_cancer_Y, test_size=0.3, random_state=0)
breast_cancer_X_train, breast_cancer_X_test = scale_data(breast_cancer_X_train, breast_cancer_X_test)

breast_cancer_best_tree = run_KFold_CV(breast_cancer_X_train, breast_cancer_y_train)
test_best_tree(breast_cancer_best_tree, breast_cancer_X_test, breast_cancer_y_test)

,Max_Tree_Depth,Average_Accuracy,Average_Node_Count
0,1.0,0.909543,3.000000
1,2.0,0.907131,7.000000
2,3.0,0.929670,14.000000
3,4.0,0.924732,21.000000
4,5.0,0.922207,27.000000
5,6.0,0.919720,29.000000
6,7.0,0.922169,29.666667
7,8.0,0.912219,30.333333
8,9.0,0.909619,30.333333
9,10.0,0.914518,30.333333


The best classifier was found with depth  1
The best classifier achieved an accuracy of  0.8947368421052632  on the test data


### Digit Recognition

In [226]:
# Digit recognition
#manipulate data into (samples, feature) matrix
digits = datasets.load_digits()
digits_n_samples = len(digits.images)
digits_X = digits.images.reshape((digits_n_samples), -1)
digits_y = digits.target

digits_X_train, digits_X_test, digits_y_train, digits_y_test = train_test_split(digits_X, digits_y, test_size=0.3, random_state=0)
digits_X_train, digits_X_test = scale_data(digits_X_train, digits_X_test)

digits_best_tree = run_KFold_CV(digits_X_train, digits_y_train)
test_best_tree(digits_best_tree, digits_X_test, digits_y_test)


,Max_Tree_Depth,Average_Accuracy,Average_Node_Count
0,1.0,0.189333,3.000000
1,2.0,0.316648,7.000000
2,3.0,0.460652,15.000000
3,4.0,0.640385,30.333333
4,5.0,0.709604,53.666667
5,6.0,0.772454,84.666667
6,7.0,0.816238,125.333333
7,8.0,0.830538,166.333333
8,9.0,0.843271,200.000000
9,10.0,0.842500,220.000000


The best classifier was found with depth  4
The best classifier achieved an accuracy of  0.65  on the test data


## 2 Decision trees vs linear models


## 3 Regression on mixed data types